In [2]:
import numpy as np
import os
import questplus
import itertools

In [10]:
subject_id = "001"
session_id = "07"
data_path = os.path.join("..", "..", "data", "raw")
session_to_load = "all" # or "last"
n_rows = [1, 12]
orientations_degrees = [0, 90]

displacement_range_cols = [-3, 3] # columns
displacement_step_cols = 0.25 # columns
displacements_cols = np.arange(displacement_range_cols[0], displacement_range_cols[1] + displacement_step_cols, displacement_step_cols)

mean_range_cols = [-2.5, 2.5] # columns
mean_step_cols = 0.25 # columns
means_cols = np.arange(mean_range_cols[0], mean_range_cols[1] + mean_step_cols, mean_step_cols)
std_range_cols = [0.1, 5.0] # columns
std_step_cols = 0.1 # columns
stds_cols = np.arange(std_range_cols[0], std_range_cols[1] + std_step_cols, std_step_cols)
lapse_rates = np.array([0., 0.05, 0.15, 0.2,  0.25, 0.3, 0.35, 0.4, 0.45])

In [4]:
def get_key(n_row, orientation_degrees):
    # % 90 to combine staircase inference for symmetrically oriented textures
    return f"n_row={n_row}_orientation_degrees={orientation_degrees % 90}"

In [11]:
staircases = {}
conditions = list(itertools.product(n_rows, orientations_degrees))
for n, o in conditions:
    assert 0 <= o < 180, "Orientations must be in [0, 180) degree range."
    key = get_key(n, o)
    if key not in staircases:
        staircases[key] = questplus.qp.QuestPlus(
            stim_domain = {"intensity": displacements_cols}, # displacement min & max
            param_domain = {"mean": means_cols, "sd": stds_cols, "lapse_rate": lapse_rates},
            outcome_domain = {"response": [1, 0]},
            func = "norm_cdf_2", # assumes lapse rate == lower asymptote
            stim_scale = "linear"
        )
print(staircases)

for root, _, files in os.walk(data_path):
    directory_name = os.path.basename(root) 
    last_session_id_string = f"{int(session_id)-1:02d}"
    if (session_to_load == "all" and directory_name[:5] == f"ID{subject_id}") or (session_to_load == "last" and directory_name[:8] == f"ID{subject_id}S{last_session_id_string}"):
        for key, staircase in staircases.items():
            for file in files: 
                if file == "questplus_staircase_" + key + ".json":
                    # Load previous stimulus response pairs
                    file_path = os.path.join(root, file)
                    previous_stimuli = []
                    previous_responses = []
                    with open(file_path, 'r') as f:  # Open in text mode
                        text = f.read()  # Load JSON data
                        previous_staircase = questplus.qp.QuestPlus.from_json(text)
                        previous_stimuli = previous_staircase.stim_history
                        previous_responses = previous_staircase.resp_history
                    
                    # Update staircase
                    print(f"Updating staircase {key} with data {directory_name}/{file}")
                    for stim, resp in zip(previous_stimuli, previous_responses):
                        # print(stim, resp)
                        staircase.update(stim = stim, outcome = resp)

{'n_row=1_orientation_degrees=0': <questplus.qp.QuestPlus object at 0x000001DF4D23DA90>, 'n_row=12_orientation_degrees=0': <questplus.qp.QuestPlus object at 0x000001DF4D5C7990>}
Updating staircase n_row=1_orientation_degrees=0 with data ID001S01_Rothe_Kai_2025-12-24_15h53.55.773/questplus_staircase_n_row=1_orientation_degrees=0.json
150
Updating staircase n_row=12_orientation_degrees=0 with data ID001S01_Rothe_Kai_2025-12-24_15h53.55.773/questplus_staircase_n_row=12_orientation_degrees=0.json
150
Updating staircase n_row=1_orientation_degrees=0 with data ID001S02_Rothe_Kai_2025-12-24_16h48.09.730/questplus_staircase_n_row=1_orientation_degrees=0.json
170
Updating staircase n_row=12_orientation_degrees=0 with data ID001S02_Rothe_Kai_2025-12-24_16h48.09.730/questplus_staircase_n_row=12_orientation_degrees=0.json
170
Updating staircase n_row=1_orientation_degrees=0 with data ID001S03_Rothe_Kai_2026-01-05_20h22.26.755/questplus_staircase_n_row=1_orientation_degrees=0.json
151
Updating stai

In [7]:
# Test
print(staircases[get_key(n_rows[0], orientations_degrees[0])].stim_history)
print(len(staircases[get_key(n_rows[0], orientations_degrees[0])].resp_history))

print(staircases[get_key(n_rows[1], orientations_degrees[0])].stim_history)
print(len(staircases[get_key(n_rows[1], orientations_degrees[0])].resp_history))

[OrderedDict([('intensity', 0.0)]), OrderedDict([('intensity', 0.5)]), OrderedDict([('intensity', 0.75)]), OrderedDict([('intensity', 0.75)]), OrderedDict([('intensity', -1.0)]), OrderedDict([('intensity', 1.5)]), OrderedDict([('intensity', -1.25)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', -1.75)]), OrderedDict([('intensity', -2.0)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', -0.5)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', -0.75)]), OrderedDict([('intensity', -1.0)]), OrderedDict([('intensity', -1.0)]), OrderedDict([('intensity', -1.0)]), OrderedDict([('intensity', 3.0)]), OrderedDict([('intensity', -1.0)]), OrderedDict([('intensity', 3.0)]), Orde

In [14]:
ID = "01"
session_id = f"{int(ID):02d}"
session_id

'01'